In [27]:
# general purpose modules for handling data
import numpy as np
from numpy import array
import pandas as pd

# for loading telo data column containing individual
# telomere length values
from ast import literal_eval

# custom module for handling telomere length data
import telomere_methods_astros as telo_ma

In [28]:
import importlib
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
&nbsp; 

The dataframe containing our telomere length data already contains the means. However, we're interested in analyzing (graphing, stats) the individual telomere length measurements. To do so we must extract the values from the list, while keeping the values associated with their timepont/individual. We'll achieve this by 'exploding' the cell containing individual telomere length measurements as a list, into one row per measurement (5520 rows per sample, 184 telos per 30 metaphases per sample). 

As well, we're interested in defining and examining the prevalence of short/long telomeres; this is achieved by finding the values that divide the data into bottom 25% (short telos), middle 50%, and top 25% (long telos). More on that soon.
 
&nbsp;
___

# Transforming telomere length data (teloFISH)

In [29]:
astro_df = pd.read_csv('../data/compiled data/All_astronauts_telomere_length_dataframe.csv')

# literal eval enables interpretation of individual telomere length values in the list 
astro_df['telo data'] = astro_df['telo data'].apply(lambda row: np.array(literal_eval(row)))

In [30]:
astro_df.head(4)

,astro number,astro id,timepoint,flight status,telo data,telo means,Q1,Q2-3,Q4
0,1,5163,L-270,Pre-Flight,"[80.90544604076177, 73.52155028399599, 91.6471...",87.672120,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
1,1,5163,L-180,Pre-Flight,"[61.777480788506516, 68.55997327096559, 190.51...",101.077756,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
2,1,5163,L-60,Pre-Flight,"[133.74540594721014, 48.34614099565653, 132.52...",128.599235,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75
3,1,5163,FD90,Mid-Flight,"[74.32342131640495, 75.1252923488139, 92.48245...",101.183129,telos preF Q1 <0.25,telos preF Q2-3 >0.25 & <0.75,telos preF Q4 >0.75


## Exploding list containing individual telomere length measurements into rows

this action explodes the list into columns bearing each datapoint.
picture the list, which contains the individual values, expanding to the
right, up to 5520 measurements

importantly, the index #s per row still refer to the timepont/sample
so we can merge these columns back to the original astro_df
then we'll melt the columns into one, resulting in a lot of rows
where each has one individual telomere length measurement

In [31]:
explode_telos_raw = astro_df['telo data'].apply(pd.Series)

In [32]:
explode_telos_raw.head(2)

,0,1,2,3,4,5,6,7,8,9,...,5510,5511,5512,5513,5514,5515,5516,5517,5518,5519
0,80.905446,73.521550,91.647177,59.839626,127.647845,123.905780,106.682259,58.954227,90.661544,58.202472,...,79.218176,73.838958,46.224524,123.671901,112.378884,85.683261,95.172068,89.508854,82.091547,72.970264
1,61.777481,68.559973,190.511193,113.448045,142.783161,71.049115,86.835951,160.992315,126.194454,85.950551,...,84.079519,92.616104,143.818911,188.439693,83.995991,96.759105,110.958904,67.056465,69.462078,120.998998


In [33]:
exploded_telos_astro_df = (explode_telos_raw
    # merge 5520 columns with original dataframe
    .merge(astro_df, right_index = True, left_index = True)
                           
    # drop unnecessary columns
    .drop(['telo data', 'Q1', 'Q2-3', 'Q4'], axis = 1)
                           
    #specify which columns remain constant per indviidual telo 
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data exploded") 
    .drop("variable", axis = 1)
    .dropna())

exploded_telos_astro_df.shape

(436080, 6)

In [69]:
copy_exploded_telos_astros_df = exploded_telos_astro_df

copy_exploded_telos_astros_df.to_csv('../data/compiled and processed data/exploded_telos_astros_df.csv', index = False)

## Loading astro telomere length data per cell, exploding cell data, saving to file

In [116]:
astro_cells_df = pd.read_csv('../data/compiled data/All_astronauts_telomere_length_per_cell_dataframe.csv')

astro_cells_df['telo data per cell'] = astro_cells_df['telo data per cell'].apply(lambda row: np.array(literal_eval(row)))
astro_cells_df['telo data per cell'][0].shape

(30,)

In [117]:
explode_cells_raw = astro_cells_df['telo data per cell'].apply(pd.Series)

In [120]:
exploded_cells_astro_df = (explode_cells_raw

    .merge(astro_cells_df, right_index = True, left_index = True)
    .drop(['telo data per cell'], axis = 1)
    .melt(id_vars = ['astro number', 'astro id', 'timepoint', 'flight status', 'telo means'], value_name = "telo data per cell") 
    .drop("variable", axis = 1)
    .dropna())

exploded_cells_astro_df[
    (exploded_cells_astro_df['astro id'] == 5163) &
    (exploded_cells_astro_df['timepoint'] == 'L-270')].shape

(30, 6)

In [121]:
exploded_cells_astro_df.to_csv('../data/compiled and processed data/exploded_cells_astros_df.csv', index = False)

## Loading astro controls telomere length data per cell, exploding cell data, saving to file

In [127]:
ctrl_cells_df = pd.read_csv('../data/compiled data/All_controls_telomere_length_per_cell_dataframe.csv')

ctrl_cells_df['telo data per cell'] = ctrl_cells_df['telo data per cell'].apply(lambda row: np.array(literal_eval(row)))
ctrl_cells_df['telo data per cell'][0].shape

(30,)

In [132]:
explode_cells_ctrl_raw = ctrl_cells_df['telo data per cell'].apply(pd.Series)
ctrl_cells_df.head(4)

,control id,timepoint,flight status controls,telo data per cell,telo means
0,100,L-270,Pre-Flight,"[0.8477957561098561, 1.0075334075822138, 0.858...",1.013536
1,100,L-180,Pre-Flight,"[1.3042379742777852, 1.2996318440034278, 1.366...",1.268621
2,100,L-60,Pre-Flight,"[1.1800698226000297, 1.0767199987117924, 1.087...",0.909948
3,100,R+60,Post-Flight,"[1.5899987895105754, 1.0877994067201529, 1.071...",1.321577


In [134]:
exploded_cells_ctrl_df = (explode_cells_ctrl_raw

    .merge(ctrl_cells_df, right_index = True, left_index = True)
    .drop(['telo data per cell'], axis = 1)
    .melt(id_vars = ['control id', 'timepoint', 'flight status controls', 'telo means'], value_name = "telo data per cell") 
    .drop("variable", axis = 1)
    .dropna())

exploded_cells_ctrl_df

In [136]:
exploded_cells_ctrl_df['flight status'] = 'Controls'

In [137]:
exploded_cells_ctrl_df.to_csv('../data/compiled and processed data/exploded_cells_ctrl_df.csv', index = False)

## Defining short/mid/long telomeres w/ quartiles and counting per timepoint per astro

This function identifies the timepoint per astronaut most distal to spaceflight (L-270 or L-180) and identifies the individual telomere length values which divide the data into the bottom 25% (short telos), mid 50%, and top 25% (long telos). The function then counts how many telos reside in each quartile. Now we have a way to define short/long telomeres.

The function then applies those quartile cutoff values to subsequent datapoints and counts how many telomeres reside within each quartile. In doing so, we count the number of telomeres moving into/out of the quartiles, per timepoint per astronaut, and can quantitatively dicuss # of short/long telos.

In [97]:
quartiles_astro_df = telo_ma.make_quartiles_columns(astro_df)

In [99]:
for col in ['Q1', 'Q2-3', 'Q4']:
    quartiles_astro_df[col] = quartiles_astro_df[col].astype('int64')

quartiles_astro_df[quartiles_astro_df['astro id'] == 2171]

astro number       int64
astro id           int64
timepoint         object
flight status     object
telo data         object
telo means       float64
Q1                 int64
Q2-3               int64
Q4                 int64
dtype: object

In [75]:
melted_quartiles_astro_df = pd.melt(
    quartiles_astro_df,
    id_vars = [col for col in quartiles_astro_df.columns if col != 'Q1' and col != 'Q2-3' and col != 'Q4'],
    var_name='relative Q',
    value_name='Q freq counts')

melted_quartiles_astro_df['Q freq counts'] = melted_quartiles_astro_df['Q freq counts'].astype('int64')
melted_quartiles_astro_df['astro id'] = melted_quartiles_astro_df['astro id'].astype('str')

In [101]:
melted_quartiles_astro_df[melted_quartiles_astro_df['astro id'] == '5163'].head(8)

,astro number,astro id,timepoint,flight status,telo data,telo means,relative Q,Q freq counts
0,1,5163,L-270,Pre-Flight,"[80.90544604076177, 73.52155028399599, 91.6471...",87.672120,Q1,1384
1,1,5163,L-180,Pre-Flight,"[61.777480788506516, 68.55997327096559, 190.51...",101.077756,Q1,848
2,1,5163,L-60,Pre-Flight,"[133.74540594721014, 48.34614099565653, 132.52...",128.599235,Q1,259
3,1,5163,FD90,Mid-Flight,"[74.32342131640495, 75.1252923488139, 92.48245...",101.183129,Q1,472
4,1,5163,FD140,Mid-Flight,"[114.1162712996993, 95.94052789842966, 85.5997...",129.851970,Q1,203
5,1,5163,R+7,Post-Flight,"[89.04109589041096, 103.97594386902773, 41.296...",82.169298,Q1,1876
6,1,5163,R+60,Post-Flight,"[66.6388239224858, 112.32876712328768, 69.4119...",100.116703,Q1,566
7,1,5163,R+180,Post-Flight,"[94.92148346140996, 62.712996992983626, 111.39...",115.403190,Q1,936


In [77]:
copy_melted_quartiles_astro_df = melted_quartiles_astro_df
copy_melted_quartiles_astro_df['telo data'] = copy_melted_quartiles_astro_df['telo data'].apply(lambda row: row.tolist())

melted_quartiles_astro_df.to_csv('../data/compiled and processed data/melted_quartiles_astro_df.csv', index = False)

# Cleaning astronaut urine biochem data (n=11)

In [138]:
urine_biochem_data = pd.read_csv('../data/compiled data/urine_biochem_data.csv')
urine_biochem_data

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0 day 1,R+0 day 2,R+30 day 1,R+30 day 2,sample type
0,"8-OHdG, Urine ug/gCr",2.6 ± 1,3.1 ± 1,3 ± 1,3.6 ± 1.2,3 ± 1,2.5 ± 0.6,2.4 ± 0.8,2.4 ± 1,1.7 ± 0.7,1.8 ± 1,urine
1,"Copper, Urine umol/day",0.28 ± 0.17,0.24 ± 0.04,0.19 ± 0.02,0.24 ± 0.04,0.21 ± 0.05,0.24 ± 0.08,0.58 ± 0.71,0.25 ± 0.07,0.26 ± 0.1,0.34 ± 0.32,urine
2,PGF2-alpha ng/mg Cr,1.84 ± 1.2,2.22 ± 0.94,2.48 ± 1.05,2.92 ± 1.38,2.7 ± 1.26,3.12 ± 1.14,1.53 ± 1.06,1.59 ± 0.91,1.88 ± 1.12,1.72 ± 1.11,urine
3,"Selenium, Urine umol/day",1.02 ± 0.39,1.11 ± 0.35,1.08 ± 0.3,1.26 ± 0.54,1.14 ± 0.42,1.19 ± 0.42,0.88 ± 0.21,0.94 ± 0.27,0.96 ± 0.31,1.03 ± 0.41,urine


In [139]:
melt_urine_biochem_data = pd.melt(
    urine_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement'
)

melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39


In [140]:
def make_flight_status_via_timepoint(row):
    if 'Pre' in row:
        return 'Pre-Flight'
    elif 'L' in row:
        return 'Pre-Flight'
    elif 'FD' in row:
        return 'Mid-Flight'
    elif 'R' in row:
        return 'Post-Flight'

In [141]:
def grab_number_remove_plusminus(row):
    if '±' in str(row):
        row = row.split('±')[0].strip()
        row = float(row)
        return row  

In [142]:
melt_urine_biochem_data['flight status'] = melt_urine_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))

In [143]:
melt_urine_biochem_data['measured analyte'] = melt_urine_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_urine_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,"8-OHdG, Urine ug/gCr",urine,Pre,2.6 ± 1,Pre-Flight,2.60
1,"Copper, Urine umol/day",urine,Pre,0.28 ± 0.17,Pre-Flight,0.28
2,PGF2-alpha ng/mg Cr,urine,Pre,1.84 ± 1.2,Pre-Flight,1.84
3,"Selenium, Urine umol/day",urine,Pre,1.02 ± 0.39,Pre-Flight,1.02


### Reading in astronaut blood biochem data (n=11)

In [84]:
blood_biochem_data = pd.read_csv('../data/compiled data/blood_biochem_data.csv')
blood_biochem_data.head(6)

,biochemistry,Pre,FD15,FD30,FD60,FD120,FD180,R+0,R+180,sample type
0,CCL2/MCP-1 pg/ml,83 ± 17,89 ± 21,94 ± 17,96 ± 26,96 ± 25,93 ± 22,182 ± 146,87 ± 23,blood
1,CCL3/MIP-1a pg/ml,438 ± 194,644 ± 214,652 ± 129,642 ± 218,489 ± 89,641 ± 247,310 ± 96,388 ± 105,blood
2,CCL4/MIP1B pg/ml,54 ± 17,75 ± 27,101 ± 34,75 ± 34,75 ± 37,70 ± 28,51 ± 16,48 ± 14,blood
3,CCL5/RANTES pg/ml,6902 ± 4393,15462 ± 2388,17986 ± 601,15022 ± 2726,14227 ± 2755,13636 ± 3964,3890 ± 3563,5569 ± 2539,blood
4,CXCL5/ENA-78 pg/ml,275 ± 549,1233 ± 967,1005 ± 426,1144 ± 823,1009 ± 503,919 ± 501,93 ± 74,134 ± 70,blood
5,CXCL8/IL-8 pg/ml,12 ± 5,23 ± 17,14 ± 7,18 ± 6,12 ± 2,19 ± 9,*,*,blood


In [85]:
blood_biochem_data.replace('*', np.NaN, inplace=True)

In [86]:
melt_blood_biochem_data = pd.melt(
    blood_biochem_data,
    id_vars=['biochemistry', 'sample type'],
    var_name='timepoint',
    value_name='measurement')

In [87]:
melt_blood_biochem_data['flight status'] = melt_blood_biochem_data['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))
melt_blood_biochem_data['measured analyte'] = melt_blood_biochem_data['measurement'].apply(lambda row: grab_number_remove_plusminus(row))
melt_blood_biochem_data.head(4)

,biochemistry,sample type,timepoint,measurement,flight status,measured analyte
0,CCL2/MCP-1 pg/ml,blood,Pre,83 ± 17,Pre-Flight,83.0
1,CCL3/MIP-1a pg/ml,blood,Pre,438 ± 194,Pre-Flight,438.0
2,CCL4/MIP1B pg/ml,blood,Pre,54 ± 17,Pre-Flight,54.0
3,CCL5/RANTES pg/ml,blood,Pre,6902 ± 4393,Pre-Flight,6902.0


In [88]:
melt_blood_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)
melt_urine_biochem_data.rename(columns={'biochemistry':'biochemistry analyte'}, inplace=True)

In [89]:
melt_urine_biochem_data.to_csv('../data/compiled and processed data/melt_urine_biochem_data.csv', index=False)
melt_blood_biochem_data.to_csv('../data/compiled and processed data/melt_blood_biochem_data.csv', index=False)

### Reading in astronaut blood biochem data (n=4)

In [90]:
blood_biochem_data_n4 = pd.read_csv('../data/compiled data/blood_biochem_data_n4.csv')
blood_biochem_data_n4.head(4)

,timepoint,CCL2/MCP-1 pg/ml,CCL3/MIP-1a pg/ml,CCL4/MIP1B pg/ml,CCL5/RANTES pg/ml,CXCL5/ENA-78 pg/ml,CXCL8/IL-8 pg/ml,Ferritin ng/mL,FGF Basic Immune Lab pg/ml,"Folate, RBC ng/ml (w/serum folat",...,Total Antiox Cap mmol/L,Total Lipid Peroxides umol/L,Tpo (Thrombopoietin) Immune Lab pg/ml,Transferrin mg/dL,TRF Receptors ug/mL,VEGF-1 Immune Lab pg/ml,Vitamin B12 pmol/L,Vitamin C ug/mL,"Zinc, Serum umol/L",astro id
0,L-180,84.73,207.7,39.7,485,14.5,<7.60,71,56.4,288.86,...,1.56,0.29,383.5,255,5,21,207.32,9.0,17.90,1261
1,L-45,84,354.1,41.9,3140,49.7,<7.60,72,69,326.51,...,1.46,0.33,477.6,223,4.6,45.5,198.47,8.0,17.44,1261
2,L-10,89.64,462.2,46.7,6345.5,179.5,<7.60,64,78.7,NaN,...,1.55,NaN,597.7,240,4.7,69.3,186.66,NaN,15.91,1261
3,FD15,76.93,846.3,63.4,>7694,575.9,18.93,89,120.1,NaN,...,1.55,NaN,1190.5,244,5,175.4,216.18,NaN,16.83,1261


In [91]:
blood_biochem_data_n4 = blood_biochem_data_n4.replace(to_replace='DNR', value=np.nan)

In [92]:
def remove_arrows(x):
    if '>' in str(x):
        x = str(x).replace('>', '').strip()
        return float(x)
    
    elif '<' in str(x):
        x = str(x).replace('<', '').strip()
        return float(x)
    
    else:
        return x

In [93]:
for col in blood_biochem_data_n4.columns:
    blood_biochem_data_n4[col] = blood_biochem_data_n4[col].apply(lambda x: remove_arrows(x))
    
blood_biochem_data_n4['flight status'] = blood_biochem_data_n4['timepoint'].apply(lambda row: make_flight_status_via_timepoint(row))

In [94]:
blood_biochem_data_n4.head(4)

,timepoint,CCL2/MCP-1 pg/ml,CCL3/MIP-1a pg/ml,CCL4/MIP1B pg/ml,CCL5/RANTES pg/ml,CXCL5/ENA-78 pg/ml,CXCL8/IL-8 pg/ml,Ferritin ng/mL,FGF Basic Immune Lab pg/ml,"Folate, RBC ng/ml (w/serum folat",...,Total Lipid Peroxides umol/L,Tpo (Thrombopoietin) Immune Lab pg/ml,Transferrin mg/dL,TRF Receptors ug/mL,VEGF-1 Immune Lab pg/ml,Vitamin B12 pmol/L,Vitamin C ug/mL,"Zinc, Serum umol/L",astro id,flight status
0,L-180,84.73,207.7,39.7,485,14.5,7.6,71,56.4,288.86,...,0.29,383.5,255,5,21,207.32,9.0,17.90,1261,Pre-Flight
1,L-45,84,354.1,41.9,3140,49.7,7.6,72,69,326.51,...,0.33,477.6,223,4.6,45.5,198.47,8.0,17.44,1261,Pre-Flight
2,L-10,89.64,462.2,46.7,6345.5,179.5,7.6,64,78.7,NaN,...,NaN,597.7,240,4.7,69.3,186.66,NaN,15.91,1261,Pre-Flight
3,FD15,76.93,846.3,63.4,7694,575.9,18.93,89,120.1,NaN,...,NaN,1190.5,244,5,175.4,216.18,NaN,16.83,1261,Mid-Flight


In [95]:
melt_blood_biochem_data_n4 = pd.melt(
    blood_biochem_data_n4,
    id_vars=['astro id', 'timepoint', 'flight status'],
    var_name='biochemistry analyte',
    value_name='measured analyte')

In [96]:
melt_blood_biochem_data_n4.to_csv('../data/compiled and processed data/melt_blood_biochem_data_n4.csv', index=False)